In [ ]:
%load_ext autoreload
%autoreload 2
from config import username, password
print(password)
from modules.login_module import logIn, launch_to_homescreen, create_directory
from modules.download_files_module import request_report_process, download_loop_missing, download_process,  move_files_over, unzip_xlsx_file, unzip_files_in_same_dir, move_xlsx_files
from modules.unit_testing import TestFileProcessing
from modules.data_transformation import *
from modules.post_download_change import *


from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException, NoSuchWindowException
from selenium.webdriver.common.action_chains import ActionChains
import os
import pandas as pd
import logging
import time
from datetime import datetime
from modules import sql_query_module
import urllib
import sqlalchemy
today_date = datetime.now()
formatted_month_day_year = today_date.strftime("%m_%d_%y")
pd.set_option('display.max_columns', None)

logging.basicConfig(filename='ELPAC_SBAC_results.log', level=logging.INFO,
                   format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S',force=True)
logging.info('\n\n-------------ELPAC_SBAC_results new instance log')

#create file_download dir, and establish download directory
create_directory('file_downloads')
create_directory(f'file_downloads\\sbac\\{formatted_month_day_year}')
create_directory(f'file_downloads\\elpac\\{formatted_month_day_year}')

# Set up Chrome options
download_directory = os.getcwd() + f'\\file_downloads\\elpac\\{formatted_month_day_year}'
chrome_options = webdriver.ChromeOptions()
prefs = {'download.default_directory' : download_directory,
         'profile.default_content_setting_values.automatic_downloads': 1,
         'profile.content_settings.exceptions.automatic_downloads.*.setting': 1}
chrome_options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(ChromeDriverManager().install(), options = chrome_options)

logIn(username, password, driver)
launch_to_homescreen(driver)

coord_list = ['LEA CAASPP Coordinator at Alain Leroy Locke College Preparatory Academy',
 'LEA CAASPP Coordinator at Animo City of Champions Charter High',
 'LEA CAASPP Coordinator at Animo Compton Charter',
 'LEA CAASPP Coordinator at Animo Ellen Ochoa Charter Middle',
 'LEA CAASPP Coordinator at Animo Florence-Firestone Charter Middle',
 'LEA CAASPP Coordinator at Animo Inglewood Charter High',
 'LEA CAASPP Coordinator at Animo Jackie Robinson High',
 'LEA CAASPP Coordinator at Animo James B. Taylor Charter Middle',
 'LEA CAASPP Coordinator at Animo Jefferson Charter Middle',
 'LEA CAASPP Coordinator at Animo Leadership High',
 'LEA CAASPP Coordinator at Animo Legacy Charter Middle',
 'LEA CAASPP Coordinator at Animo Mae Jemison Charter Middle',
 'LEA CAASPP Coordinator at Animo Pat Brown',
 'LEA CAASPP Coordinator at Animo Ralph Bunche Charter High',
 'LEA CAASPP Coordinator at Animo South Los Angeles Charter',
 'LEA CAASPP Coordinator at Animo Venice Charter High',
 'LEA CAASPP Coordinator at Animo Watts College Preparatory Academy',
 'LEA CAASPP Coordinator at Oscar De La Hoya Animo Charter High',
 'LEA ELPAC Coordinator at Alain Leroy Locke College Preparatory Academy',
 'LEA ELPAC Coordinator at Animo City of Champions Charter High',
 'LEA ELPAC Coordinator at Animo Compton Charter',
 'LEA ELPAC Coordinator at Animo Ellen Ochoa Charter Middle',
 'LEA ELPAC Coordinator at Animo Florence-Firestone Charter Middle',
 'LEA ELPAC Coordinator at Animo Inglewood Charter High',
 'LEA ELPAC Coordinator at Animo Jackie Robinson High',
 'LEA ELPAC Coordinator at Animo James B. Taylor Charter Middle',
 'LEA ELPAC Coordinator at Animo Jefferson Charter Middle',
 'LEA ELPAC Coordinator at Animo Leadership High',
 'LEA ELPAC Coordinator at Animo Legacy Charter Middle',
 'LEA ELPAC Coordinator at Animo Mae Jemison Charter Middle',
 'LEA ELPAC Coordinator at Animo Pat Brown',
 'LEA ELPAC Coordinator at Animo Ralph Bunche Charter High',
 'LEA ELPAC Coordinator at Animo South Los Angeles Charter',
 'LEA ELPAC Coordinator at Animo Venice Charter High',
 'LEA ELPAC Coordinator at Animo Watts College Preparatory Academy',
 'LEA ELPAC Coordinator at Oscar De La Hoya Animo Charter High']

elpac_coordinators = [coord for coord in coord_list if 'ELPAC' in coord]
caaspp_coordinators = [coord for coord in coord_list if 'CAASPP' in coord]

#This exists when passing names into the requested reports, as a subset. Change list into a set to only retain unique schools
school_report_names = [entry.split(' at ', 1)[1] for entry in elpac_coordinators]
school_report_names = list(set(school_report_names))

# ---------------------------------------SBAC Files Request and Download

# Call the function, school report names variable is called for just school name
#Equivalent of Student Score Data File
request_report_process(driver, 'SBAC', 'CAASPP_Student_Score_Data_Extract_Report', caaspp_coordinators, '2023')
download_process(school_report_names, '2023 CAASPP Student Score Data File By Enrolled LEA', driver) 

#This is here three times to see if anything got skipped the first time. Initial dir is set at ELPAC only to move the files over to SBAC dir
#Will run 5 times

time.sleep(10) #implemented to give time for files to download
download_loop_missing(f'elpac\\{formatted_month_day_year}', '2023 CAASPP Student Score Data File By Enrolled LEA', driver)

#This moves the files from ELPAC  timestamp dir to SBAC timestamp dir. 
#This is because the download dir cannot be changed in Selenium
move_files_over()

# --------------------------------------------ELPAC Files Request and Download

driver.switch_to.default_content()
request_report_process(driver, 'ELPAC', 'Student_Results_Report_Student_Score_Data_Extract', elpac_coordinators, '2023')
download_process(school_report_names, '2023 Summative ELPAC and Summative Alternate ELPAC Student Score Data File By Enrolled LEA', driver) 

time.sleep(10) #implemented to give time for files to download
#This is here three times to see if anything got skipped the first time. 
#Dir remains ELPAC for constant download directory
download_loop_missing(f'elpac\\{formatted_month_day_year}', '2023 Summative ELPAC and Summative Alternate ELPAC Student Score Data File By Enrolled LEA', driver)

#Close out driver window once done
driver.close()

#Takes 14 mins to run up to this point
# -----------------------------------------Unzip the Files and Move them to the P-Drive in this location 'P:\Knowledge Management\Ellevation\Data Sent 2023-24\State Testing'
unzip_files_in_same_dir('elpac')
unzip_files_in_same_dir('sbac')

#Keeps raw zip files in the same dir. Only moves over xlsx files
try:
    move_xlsx_files('sbac')
    logging.info('Moved SBAC XLSX files to p-drive')
except:
    logging.info('Unable to move SBAC XLSX files to the p-drive, must be connected to the VPN')
try:
    move_xlsx_files('elpac')
    logging.info('Moved ELPAC XLSX files to p-drive')
except:
    logging.info('Unable to move ELPAC XLSX files to the p-drive, must be connected to the VPN')


#Checks these dirs, for all files being there
test_instance = TestFileProcessing()
test_instance.test_file_processing('sbac')
test_instance.test_file_processing('elpac')

# #Takes roughly 25 mins to download and send
# #Must be connected to the p-drive

# ---------------------------------STACKING & SENDING FILES----------------------------------

directory_path = r'P:\Knowledge Management\Ellevation\Data Sent 2023-24\State Testing'
formatted_month_day_year = today_date.strftime("%m_%d_%y")


directory_path_sbac = os.path.join(directory_path, f'sbac_{formatted_month_day_year}')
sbac = stack_files(directory_path_sbac)
sbac['CALPADSSchoolCode'] = sbac['CALPADSSchoolCode'].astype(str).str[7:]


directory_path_elpac = os.path.join(directory_path, f'elpac_{formatted_month_day_year}')
elpac = stack_files(directory_path_elpac)
elpac['CALPADSSchoolCode'] = elpac['CALPADSSchoolCode'].astype(str).str[7:]

sbac = filter_on_full_cds_code(sbac, 'CALPADSSchoolCode')
elpac = filter_on_full_cds_code(elpac, 'CALPADSSchoolCode')


# -------------------------------------------get_elpac_import------------------
elpac = get_elpac_import(elpac)
elpac.name = 'ELPAC'
directory_path_elpac = os.path.join(directory_path, f'ELPAC_STACKED_{formatted_month_day_year}.csv')
try:
    elpac.to_csv(directory_path_elpac, index=False)
    logging.info(f'ELPAC sent to {directory_path} for ellevation pickup')
except:
    logging.info(f'ELPAC unable to send for ellevation pickup')


#Differing decoding method. Refer to message with Abi
# ss_decode = {4.0: 'WelDev', 
#                 3.0: 'ModDev', 
#                 2.0: 'SomDev',
#                 1.0: 'MinDev', 
#                 '': 'No Score', 
#                 'NS': 'No Score'}

# -------------------------------Get SBAC import-------------------------

#Missing PLScore Column and ProficiencyLevelCode Mapping
sbac_final = get_sbac_cols(sbac, 'SBAC')
sbac_final.name = 'SBAC'
directory_path_sbac = os.path.join(directory_path, f'SBAC_STACKED_{formatted_month_day_year}.csv')
try:
    sbac_final.to_csv(directory_path_sbac, index=False)
    logging.info(f'SBAC sent to {directory_path} for ellevation pickup')
except:
    logging.info(f'SBAC unable to send for ellevation pickup')

# PL Score 1	STNM
# PL Score 2	STNL
# PL Score 3	STMT
# PL Score 4	STEX

# ------------------------------Get CAST import-------------------------

cast = get_cast_cols(sbac, 'CAST')
cast.name = 'CAST'
directory_path_cast = os.path.join(directory_path, f'CAST_STACKED_{formatted_month_day_year}.csv')
try:
    cast.to_csv(directory_path_cast, index=False)
    logging.info(f'CAST sent to {directory_path} for ellevation pickup')
except:
    logging.info(f'CAST unable send for ellevation pickup')

# PL Score 1	BLST
# PL Score 2	ANST
# PL Score 3	ABST

# ------------------------------------------------------send to DataTeamSandbox on 89 server-----------------------------

quoted = urllib.parse.quote_plus("Driver={SQL Server Native Client 11.0};"
                     "Server=10.0.0.89;"
                     "Database=DataTeamSandbox;"
                     "Trusted_Connection=yes;")

engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))


query_cols = ['Abbreviation',  'StudentNumber', 'SSID', 'TestGrade', 'ELStatus', 'TestDate', 
            'TestType', 'testname', 'ScaleScore','PLScore', 'ProficiencyLevelCode']

blank_cols = ['SchoolID', 'MasterSchoolID', 'StudentID', 'DisplayDate', 'TestPeriod', 'TestScoreType']


def obtain_new_scores(file, query_cols, blank_cols):

    #cut down current file to non blank cols
    file = file[query_cols]

    file = file.fillna(0)
    query = query.fillna(0)
    file = file.replace('NS', 0) #one off exception

    file['ScaleScore'] = pd.to_numeric(file['ScaleScore'])
    query['ScaleScore'] = pd.to_numeric(query['ScaleScore'])

    merging_cols = ['SSID', 'TestType', 'testname', 'ScaleScore']
    merged_df = pd.merge(query, file, on=merging_cols, how='left', indicator=True,suffixes=('_query', '_file'))

    #insert blank cols

    return(merged_df)




def send_to_SQL(file):


    dtypes, table_cols = sql_query_module.SQL_query.get_dtypes(file, 'DataTeamSandbox', f'{file.name}_Scores')

    file = obtain_new_scores(file, query_cols, blank_cols)

    # elpac.to_sql(f'{file.name}_Scores', schema='dbo', con = engine, if_exists = 'replace', index = False, dtype=dtypes)
    # engine.dispose()



query = sql_query_module.SQL_query.get_new(f'{file.name}_Scores', query_cols)

send_to_SQL(elpac)
send_to_SQL(sbac_final)
send_to_SQL(cast)
engine.dispose()


#What about selecting the initial admin year when switching over Y2Y?

In [20]:
%load_ext autoreload
%autoreload 2
from config import username, password
print(password)
from modules.login_module import logIn, launch_to_homescreen, create_directory
from modules.download_files_module import request_report_process, download_loop_missing, download_process,  move_files_over, unzip_xlsx_file, unzip_files_in_same_dir, move_xlsx_files
from modules.unit_testing import TestFileProcessing
from modules.data_transformation import *
from modules.post_download_change import *
from modules.sql_query_module import *


from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException, NoSuchWindowException
from selenium.webdriver.common.action_chains import ActionChains
import os
import pandas as pd
import logging
import time
from datetime import datetime
from modules.sql_query_module import SQL_query
import urllib
import sqlalchemy
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
today_date = datetime.now()
formatted_month_day = today_date.strftime("%m_%d")
pd.set_option('display.max_columns', None)





cast = pd.read_csv('cast.csv')
cast.name = 'cast'

elpac = pd.read_csv('elpac.csv')
elpac.name = 'elpac'

sbac_final = pd.read_csv('sbac.csv')
sbac_final.name = 'sbac'



query_cols = ['Abbreviation',  'StudentNumber', 'SSID', 'TestGrade', 'ELStatus', 'TestDate', 
            'TestType', 'testname', 'ScaleScore','PLScore', 'ProficiencyLevelCode']

blank_cols = ['SchoolID', 'MasterSchoolID', 'StudentID', 'DisplayDate', 'TestPeriod', 'TestScoreType']


def clean_up_rotating_file(file, query_cols):

    #Clean up the given file to prepare for a merge
    file = file[query_cols]
        # Replace 'NS' and NaN values with a placeholder value
    file['ScaleScore'] = file['ScaleScore'].replace(['NS', np.nan], 0)

    # Convert the column to float
    file['ScaleScore'] = file['ScaleScore'].astype(float)



def obtain_new_scores(file, file_name,  query_cols, blank_cols):


    # file = file.fillna(0)

    # query = SQL_query.get_new(file_name, query_cols)

    # query = query.fillna(0)
    # file = file.replace('NS', 0) #one off exception

    # file['ScaleScore'] = pd.to_numeric(file['ScaleScore'])
    # query['ScaleScore'] = pd.to_numeric(query['ScaleScore'])

    # merging_cols = ['SSID', 'TestType', 'testname', 'ScaleScore']
    # merged_df = pd.merge(query, file, on=merging_cols, how='outer', indicator=True,suffixes=('_query', '_file'))
    #If the file is left_only it means the records are only found in query
    #If the file is right_only it means the records are only found in file, hence the new records not in the db

    return(file)

    # new_records = merged_df.loc[merged_df['_merge'] == 'right_only']

    # #drop the merge col
    # new_records = new_records.drop('_merge', axis=1)

    # #Subset the frame down to _file columns
    # file_columns = [col for col in list(new_records.columns) if col.endswith("_file")]
    # new_records = new_records[file_columns]
    
    # column_mapping = {col: col.replace('_file', '') for col in new_records.columns}

    # #insert blank cols
    # for col in blank_cols:
    #     new_records[col] = np.nan

    # # Rename the columns using the mapping
    # new_records = new_records.rename(columns=column_mapping)
    # return(new_records)




def send_to_SQL(file, file_name, query_cols, blank_cols):

    dtypes, table_cols = SQL_query.get_dtypes(file, 'DataTeamSandbox', f'{file_name}_Scores')

    file = obtain_new_scores(file, file_name, query_cols, blank_cols)

    return(file)

    # if file.empty == True:
    #     logging.info(f'No new records to insert, {file_name} file is empty')
    # else:
    #     file.to_csv(f'file_downloads/{file_name}_new_records.csv')
    #     logging.info(f'{file_name}_new_records.csv sent to file downloads')
    #     try:
    #         file.to_sql(f'{file_name}_Scores', schema='dbo', con = SQL_query.engine, if_exists = 'append', index = False, dtype=dtypes)
    #         logging.info(f'Appending {len(file)} to {file_name}_Scores')

    #     except Exception as e:
    #         logging.info(f'Unable to append to table {file_name}_Scores due to \n {e}')



file = send_to_SQL(elpac, elpac.name, query_cols, blank_cols)
# send_to_SQL(sbac_final, sbac_final.name, query_cols, blank_cols)
# send_to_SQL(cast, cast.name, query_cols, blank_cols)

#Ok it keeps appending. At this point the records should be in and it should register. Could be an issue with the merge


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
efW5J$kq$Z89AG3


c:\Users\samuel.taylor\Desktop\Python_Scripts\Manual_Triggers\ELPAC_SBAC_Results_Selenium\modules\sql_query_module.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_SQL = pd.read_sql_query(query, con = conn)


In [25]:
pd.DataFrame(file['ScaleScore'].unique()).sort_values(by=0)

403	NS
# 307	NaN

,0
547,1223.0
549,1257.0
539,1283.0
550,1290.0
535,1302.0
...,...
382,676.0
335,757.0
373,847.0
403,NS


In [19]:
# file

#So the file columns are the only ones that I want. 
#Right only means the records are only in the file frame and not in the query frame. 
#This is what I want. Could be an issue with dtypes. 

# merging_cols = ['SSID', 'TestType', 'testname', 'ScaleScore']

elpac[['SSID', 'TestType', 'testname', 'ScaleScore']].head(40)

#Drop rows with no SSID, TestType, testname, ScaleScore

# TestType, TestName are strings

,SSID,TestType,testname,ScaleScore
Column,,,,
0,8302001725,ELPAC,ELPAC-Overall,1553.0
1,9090968962,ELPAC,ELPAC-Overall,1618.0
2,7765342705,ELPAC,ELPAC-Overall,1541.0
3,1671847716,ELPAC,ELPAC-Overall,1546.0
4,3527097153,ELPAC,ELPAC-Overall,1606.0
5,8584196218,ELPAC,ELPAC-Overall,1566.0
6,5375663583,ELPAC,ELPAC-Overall,1559.0
7,3680068443,ELPAC,ELPAC-Overall,1678.0
8,1081709144,ELPAC,ELPAC-Overall,1650.0


In [17]:


# Use boolean indexing to filter rows where all selected columns have 0 values
file[(file[['SSID', 'ScaleScore']] == 0.0).all(axis=1)]

,Abbreviation_query,StudentNumber_query,SSID,TestGrade_query,ELStatus_query,TestDate_query,TestType,testname,ScaleScore,PLScore_query,ProficiencyLevelCode_query,Abbreviation_file,StudentNumber_file,TestGrade_file,ELStatus_file,TestDate_file,PLScore_file,ProficiencyLevelCode_file,_merge
0,Animo South Los Angeles Charter,7.390900e+04,0.0,9.0,Yes,2024-03-31,0,0,0.0,1,MinDev,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,Animo Venice Charter High,7.050800e+04,0.0,9.0,Yes,2024-03-31,0,0,0.0,3,WelDev,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,Animo Ralph Bunche Charter High,7.046100e+04,0.0,9.0,Yes,2024-03-31,0,0,0.0,3.0,WelDev,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,Animo Ralph Bunche Charter High,6.910600e+04,0.0,10.0,Yes,2024-03-31,0,0,0.0,3.0,WelDev,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,Animo Jackie Robinson High,7.805700e+04,0.0,10.0,Yes,2024-03-31,0,0,0.0,1.0,MinDev,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32831,Animo Legacy Charter Middle,8.284200e+04,0.0,6.0,Yes,2024-03-31,0,0,0.0,0,No Score,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
32832,Animo Legacy Charter Middle,8.192400e+04,0.0,6.0,Yes,2024-03-31,0,0,0.0,0,No Score,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
32833,Animo Legacy Charter Middle,8.549742e+09,0.0,7.0,Yes,2024-03-31,0,0,0.0,0,No Score,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
32834,Animo Leadership High,8.296200e+04,0.0,9.0,Yes,2024-03-31,0,0,0.0,2,Som-ModDev,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [18]:
elpac[(elpac[['SSID', 'ScaleScore']] == 0.0).all(axis=1)]

,Abbreviation,SchoolID,MasterSchoolID,StudentNumber,StudentID,SSID,TestGrade,ELStatus,TestDate,DisplayDate,TestType,TestPeriod,TestScoreType,testname,ScaleScore,PLScore,ProficiencyLevelCode
Column,,,,,,,,,,,,,,,,,


In [5]:
elpac

,Abbreviation,SchoolID,MasterSchoolID,StudentNumber,StudentID,SSID,TestGrade,ELStatus,TestDate,DisplayDate,TestType,TestPeriod,TestScoreType,testname,ScaleScore,PLScore,ProficiencyLevelCode
Column,,,,,,,,,,,,,,,,,
0,Oscar De La Hoya Animo Charter High,NaN,NaN,70786,NaN,8302001725,9,Yes,2024-03-31,NaN,ELPAC,NaN,NaN,ELPAC-Overall,1553.0,3.0,WelDev
1,Oscar De La Hoya Animo Charter High,NaN,NaN,83613,NaN,9090968962,11,Yes,2024-03-31,NaN,ELPAC,NaN,NaN,ELPAC-Overall,1618.0,4.0,WelDev
2,Oscar De La Hoya Animo Charter High,NaN,NaN,77622,NaN,7765342705,10,Yes,2024-03-31,NaN,ELPAC,NaN,NaN,ELPAC-Overall,1541.0,2.0,Som-ModDev
3,Animo Ellen Ochoa Charter Middle,NaN,NaN,76240,NaN,1671847716,8,Yes,2024-03-31,NaN,ELPAC,NaN,NaN,ELPAC-Overall,1546.0,3.0,WelDev
4,Oscar De La Hoya Animo Charter High,NaN,NaN,77615,NaN,3527097153,10,Yes,2024-03-31,NaN,ELPAC,NaN,NaN,ELPAC-Overall,1606.0,4.0,WelDev
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9942,Animo Inglewood Charter High,NaN,NaN,81721,NaN,9762550334,9,Yes,2024-03-31,NaN,ELPAC,NaN,NaN,ELPAC-Writing,1563.0,2.0,Som-ModDev
9943,Animo Inglewood Charter High,NaN,NaN,81795,NaN,3456637885,9,Yes,2024-03-31,NaN,ELPAC,NaN,NaN,ELPAC-Writing,1570.0,2.0,Som-ModDev
9944,Animo Inglewood Charter High,NaN,NaN,75000,NaN,3736250313,11,Yes,2024-03-31,NaN,ELPAC,NaN,NaN,ELPAC-Writing,1645.0,3.0,WelDev


In [36]:
file

#retain all the file columns

#Doing a left join on the file is an issue. Because the new incoming records will not match up. 
#So need to make it an outer join

,Abbreviation,StudentNumber,TestGrade,ELStatus,TestDate,PLScore,ProficiencyLevelCode,SchoolID,MasterSchoolID,StudentID,DisplayDate,TestPeriod,TestScoreType
1763,Oscar De La Hoya Animo Charter High,7.078600e+04,9.0,Yes,2024-03-31,3.0,WelDev,NaN,NaN,NaN,NaN,NaN,NaN
1764,Oscar De La Hoya Animo Charter High,8.361300e+04,11.0,Yes,2024-03-31,4.0,WelDev,NaN,NaN,NaN,NaN,NaN,NaN
1765,Oscar De La Hoya Animo Charter High,7.762200e+04,10.0,Yes,2024-03-31,2.0,Som-ModDev,NaN,NaN,NaN,NaN,NaN,NaN
1766,Animo Ellen Ochoa Charter Middle,7.624000e+04,8.0,Yes,2024-03-31,3.0,WelDev,NaN,NaN,NaN,NaN,NaN,NaN
1767,Oscar De La Hoya Animo Charter High,7.761500e+04,10.0,Yes,2024-03-31,4.0,WelDev,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9967,Animo Legacy Charter Middle,8.284200e+04,6.0,Yes,2024-03-31,0,No Score,NaN,NaN,NaN,NaN,NaN,NaN
9968,Animo Legacy Charter Middle,8.192400e+04,6.0,Yes,2024-03-31,0,No Score,NaN,NaN,NaN,NaN,NaN,NaN
9969,Animo Legacy Charter Middle,8.549742e+09,7.0,Yes,2024-03-31,0,No Score,NaN,NaN,NaN,NaN,NaN,NaN
9970,Animo Leadership High,8.296200e+04,9.0,Yes,2024-03-31,2,Som-ModDev,NaN,NaN,NaN,NaN,NaN,NaN


# Notes 

In [ ]:
#send this file to Abi. 
#original name mapping to new columns

#Abbreviation - map on CALPADSDIstrictName
#schoolid blank
#MasterSchoolID blank
#StudentNumber - LocalStudentId
#StudentID blank
#SSID - SSID
#TestGrade - GradeAssessed
#ELStatus Blank
#TestDate - End of the month for the month uploaded
#DisplayDate blank
#testtype - 21 (General ELPAC) 23(ALT ELPAC)
#testperiod blank
#testscoretype blank
#testname ListeningPL	SpeakingPL	ReadingPL	WritingPL 
#scalescore and plscore melt down
#proficiencylevelcode is hard mapping

#How often do you need this done?
#People dont start asking until end of the testing window, but want a monthly pull. As we lose enrollments of students, we lose access to their scores
#End of the month from Feb through May. Last day of Feb.

#Prioritize ELPAC pulldowns are Feb-May. 
#SBAC is April - June (mid) (End of April for SBAC/CAST)

#CAST has recordtypes that are taken apart within the SBAC file. 

#What table do they send it to?
# ELPAC [TestScores].[dbo].[vw_rpt_ELPACScores]
# CAST  [TestScores].[dbo].[CASTScores]
# SBAC: ELA/Math [TestScores].[dbo].[vw_rpt_SBACScores]